In [1]:
import pandas as pd
import os

In [2]:
#----- preprocessing the text data --#
#read datasets/train.csv
SalesData = pd.read_csv('Train.csv')

In [5]:
#print out first row of papers
print (SalesData.head(5))


  Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0           FDA15         9.30          Low Fat         0.016047   
1           DRC01         5.92          Regular         0.019278   
2           FDN15        17.50          Low Fat         0.016760   
3           FDX07        19.20          Regular         0.000000   
4           NCD19         8.93          Low Fat         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                       1999      Medium               Tier 1   
1                       2009      Medium               Tier 3   
2                       1999      Medium               Tier

In [6]:
print('The shape of our›› features is:', SalesData.shape)
#Descriptive statitics for each column
print (SalesData.describe())

The shape of out features is: (8523, 12)
       Item_Weight  Item_Visibility     Item_MRP  Outlet_Establishment_Year  \
count  7060.000000      8523.000000  8523.000000                8523.000000   
mean     12.857645         0.066132   140.992782                1997.831867   
std       4.643456         0.051598    62.275067                   8.371760   
min       4.555000         0.000000    31.290000                1985.000000   
25%       8.773750         0.026989    93.826500                1987.000000   
50%      12.600000         0.053931   143.012800                1999.000000   
75%      16.850000         0.094585   185.643700                2004.000000   
max      21.350000         0.328391   266.888400                2009.000000   

       Item_Outlet_Sales  
count        8523.000000  
mean         2181.288914  
std          1706.499616  
min            33.290000  
25%           834.247400  
50%          1794.331000  
75%          3101.296400  
max         13086.964800  


In [65]:
#one-hot encode the data using pandas get_dummies
features = pd.get_dummies(SalesData)
#Display the first 5 rows of 12 columns
print (features.iloc[:,:5].head(5))

   Item_Weight  Item_Visibility  Item_MRP  Outlet_Establishment_Year  \
0         9.30         0.016047  249.8092                       1999   
1         5.92         0.019278   48.2692                       2009   
2        17.50         0.016760  141.6180                       1999   
3        19.20         0.000000  182.0950                       1998   
4         8.93         0.000000   53.8614                       1987   

   Item_Outlet_Sales  
0          3735.1380  
1           443.4228  
2          2097.2700  
3           732.3800  
4           994.7052  


In [66]:
SalesData.apply(lambda x: len(x.unique()))

Item_Identifier              1559
Item_Weight                   416
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     4
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [70]:
#numpy for array conversion
import numpy as np 
#lables are values we want to predict
lables = np.asarray(features['Item_Outlet_Sales'])
#remove the labels from features
#axis 1 refers to the columns 
features = features.drop('Item_Outlet_Sales', axis = 1)
#saving feature names for later use
features_list = list(features.columns)
#convert to numpy array
features = np.array(features)

In [71]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)


In [72]:

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (6392, 1604)
Training Labels Shape: (6392,)
Testing Features Shape: (2131, 1604)
Testing Labels Shape: (2131,)


In [32]:
# The baseline are the historical averages
baseline_preds = test_features[:, features_list.index('Item_MRP')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))


Average baseline error:  1957.24


In [34]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
print("train_features")
# Train the model on training data
rf.fit(np.nan_to_num(train_features), train_labels);
# # Use the forest's predict method on the test data
predictions = rf.predict(np.nan_to_num(test_features))



train_features


In [35]:
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 771.17 degrees.
Accuracy: 43.15 %.


In [69]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export as dot file
export_graphviz(tree,
                out_file='tree.dot',
                feature_names = features_list,
                class_names = features_list,
                rounded = True, proportion = False,
                precision = 2, filled = True)
import graphviz
export_graphviz(tree, out_file="mytree.dot")
with open("mytree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)
from io import StringIO
import pydot 
pydot.graph_from_dot_data()
dot_data = StringIO()
export_graphviz(tree, out_file=dot_data)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png("dtree.png")


TypeError: graph_from_dot_data() missing 1 required positional argument: 's'